In [3]:
import PySimpleGUI as sg
import vlc
from time import sleep
import os
from sys import platform

#----------- TEST MEDIA -----------#
#  next: playlist
import test_data
song1_path = os.path.join(test_data.media_path, 'test1.mp4')
song2_path = test_data.song2    # try diff media & diff path

#----------- GUI -----------#
# next: position mult windows across multiple monitors

# how can i make this more compact?
layout1 = [
    [sg.Graph(canvas_size=(900,480), graph_bottom_left=(0,0), graph_top_right=(900,480), background_color='black', key='-G-')],
    [sg.Button('Exit', size=(10,1), pad=(0,0))]
    ]
layout2 = [
    [sg.Graph(canvas_size=(900,480), graph_bottom_left=(0,0), graph_top_right=(900,480), background_color='black', key='-G-')],
    [sg.Button('Exit', size=(10,1), pad=(0,0))]
    ]

# def btn(name):
#     return sg.Button(name, size=(10,1), pad=(0,0))

window1 = sg.Window('1one1', layout1, finalize=True, \
                    location=(0,0))

window2 = sg.Window('2two2', layout2, finalize=True,
                    location=(window1.current_location()[0]+930,window1.current_location()[1]))



print(window1.size, ' is window 1 size')
print(window1.current_location(), ' is window 1 location')
print(window2.size, ' is window 2 size')
print(window2.current_location(), ' is window 2 location')


#----------- PLAYER SETUP -----------#
# shorten this code will need N players
instance1 = vlc.Instance()
instance2 = vlc.Instance()
player1 = instance1.media_player_new()
player2 = instance2.media_player_new()
media1 = instance1.media_new(song1_path)
media2 = instance2.media_new(song2_path)
player1.set_hwnd(window1['-G-'].Widget.winfo_id())
player2.set_hwnd(window2['-G-'].Widget.winfo_id())
player1.set_media(media1)
player2.set_media(media2)


#----------- MAIN LOOP -----------#
# next: make sure media is on loop

i, paused = 0, [False, False]

while True:
    window, event, values = sg.read_all_windows(timeout=1000)
    player1.play()
    player2.play()
    if (window1 == sg.WIN_CLOSED or window2 == sg.WIN_CLOSED) \
        or (event == 'Exit'):
        break
window1.close()
window2.close()

#----------- CLEANUP -----------#
player1.stop()
player2.stop()
player1.release()
player2.release()
instance1.release()
instance2.release()



(930, 520)  is window 1 size
(0, 0)  is window 1 location
(930, 520)  is window 2 size
(930, 0)  is window 2 location
